In [ ]:
import json
import pandas as pd
import ollama
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

# --------------------------- Topics & ID mapping ---------------------------
topics = [
    'Anxiety Disorders','Depression & Mood Disorders','Stress Management','Trauma & PTSD',
    'Obsessive-Compulsive Disorder (OCD)','Relationship & Family Issues','Self-Esteem & Self-Worth Issues',
    'Addiction & Substance Use','Eating Disorders','Sleep Disorders','Suicidal Ideation & Self-Harm'
]

id2label = {i: t for i, t in enumerate(topics)}
label2id = {t: i for i, t in id2label.items()}

# --------------------------- Initialize Ollama ---------------------------
ollama_client = ollama()

# --------------------------- Predict Topic Function ---------------------------
def predict_topic(dialog, model="llama3.2-vision:latest", topics=topics):
    few_shot_examples = """
Example 1:
Dialog: "I can’t sleep at night, I keep waking up and feel exhausted during the day."
Topic: Sleep Disorders

Example 2:
Dialog: "I’ve been feeling hopeless and sad, I don’t want to get out of bed."
Topic: Depression & Mood Disorders

Example 3:
Dialog: "I keep checking the stove again and again, I can’t stop."
Topic: Obsessive-Compulsive Disorder (OCD)

Example 4:
Dialog: "I feel nervous all the time, like something bad will happen."
Topic: Anxiety Disorders
"""
    prompt = f"""
You are a mental health assistant.

{few_shot_examples}

Now classify this new dialog:
Dialog: {dialog}

Choose EXACTLY ONE topic from this list:
{', '.join(topics)}

Answer ONLY with the topic name.
"""
    result = ollama_client.generate(model=model, prompt=prompt, max_tokens=30)
    text = result["output"]

    # Match the topic in the output
    for t in topics:
        if t.lower() in text.lower():
            return t
    return "unknown"

# --------------------------- Evaluate Model Function ---------------------------
def evaluate_model(json_path, label_key="topic", cm_path="confusion_matrix.png"):
    with open(json_path, "r", encoding="utf-8") as f:
        dataset = json.load(f)

    y_true, y_pred = [], []

    for idx, sample in enumerate(dataset, start=1):
        dialog = sample["dialog"]
        true_label = sample[label_key]

        pred_label = predict_topic(dialog)

        y_true.append(label2id[true_label])
        y_pred.append(label2id[pred_label])

        status = "PASSED ✅" if pred_label == true_label else "FAILED ❌"
        print(f"Sample {idx}: Predicted={pred_label}, True={true_label} --> {status}")

    # Accuracy and classification report
    overall_acc = accuracy_score(y_true, y_pred)
    print("\n--- Classification Report ---")
    print(classification_report(y_true, y_pred, target_names=list(label2id.keys())))
    print(f"\nOverall Accuracy: {overall_acc:.4f}")

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=list(label2id.keys()),
                yticklabels=list(label2id.keys()))
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.xticks(rotation=45, ha="right")
    plt.yticks(rotation=0)
    
    plt.savefig(cm_path, dpi=300, bbox_inches="tight")
    print(f"\n✅ Confusion matrix saved as {cm_path}")
    plt.show()

    return y_true, y_pred

# --------------------------- Run Evaluation ---------------------------
if __name__ == "__main__":
    json_path = "TS_Testing.json"  # Your JSON dataset
    y_true, y_pred = evaluate_model(json_path)

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

# --------------------------- Topics & ID mapping ---------------------------
topics = [
    'Anxiety Disorders','Depression & Mood Disorders','Stress Management','Trauma & PTSD',
    'Obsessive-Compulsive Disorder (OCD)','Relationship & Family Issues','Self-Esteem & Self-Worth Issues',
    'Addiction & Substance Use','Eating Disorders','Sleep Disorders','Suicidal Ideation & Self-Harm'
]

id2label = {i: t for i, t in enumerate(topics)}
label2id = {t: i for i, t in id2label.items()}

# --------------------------- LangChain Model ---------------------------
model = OllamaLLM(model="llama3.1")

template = """
You are a mental health assistant.

Classify the following dialog into exactly ONE topic from this list:
{topic_list}

Dialog: {dialog}

Answer ONLY with the topic name.
"""

prompt = ChatPromptTemplate.from_template(template)

chain = prompt | model

# --------------------------- Predict Topic Function ---------------------------
def predict_topic(dialog, topics=topics):
    response = chain.invoke({
        "dialog": dialog,
        "topic_list": ", ".join(topics)
    })
    text = str(response)

    # Match the topic in the output
    for t in topics:
        if t.lower() in text.lower():
            return t
    return "unknown"

# --------------------------- Evaluate Model Function ---------------------------
def evaluate_model(json_path, label_key="topic", cm_path="confusion_matrix.png"):
    with open(json_path, "r", encoding="utf-8") as f:
        dataset = json.load(f)

    y_true, y_pred = [], []

    for idx, sample in enumerate(dataset, start=1):
        dialog = sample["dialog"]
        true_label = sample[label_key]

        pred_label = predict_topic(dialog)

        y_true.append(label2id[true_label])
        y_pred.append(label2id.get(pred_label, -1))  # -1 for unknown

        status = "PASSED ✅" if pred_label == true_label else "FAILED ❌"
        print(f"Sample {idx}: Predicted={pred_label}, True={true_label} --> {status}")

    # Accuracy and classification report
    overall_acc = accuracy_score(y_true, y_pred)
    print("\n--- Classification Report ---")
    print(classification_report(y_true, y_pred, target_names=list(label2id.keys()), zero_division=0))
    print(f"\nOverall Accuracy: {overall_acc:.4f}")

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=list(range(len(topics))))
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=list(label2id.keys()),
                yticklabels=list(label2id.keys()))
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.xticks(rotation=45, ha="right")
    plt.yticks(rotation=0)
    
    plt.savefig(cm_path, dpi=300, bbox_inches="tight")
    print(f"\n✅ Confusion matrix saved as {cm_path}")
    plt.show()

    return y_true, y_pred

# --------------------------- Run Evaluation ---------------------------
if __name__ == "__main__":
    json_path = "TS_Testing.json"  # Your JSON dataset
    y_true, y_pred = evaluate_model(json_path)


In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

# --------------------------- Topics ---------------------------
topics = [
    'Anxiety Disorders','Depression & Mood Disorders','Stress Management','Trauma & PTSD',
    'Obsessive-Compulsive Disorder (OCD)','Relationship & Family Issues','Self-Esteem & Self-Worth Issues',
    'Addiction & Substance Use','Eating Disorders','Sleep Disorders','Suicidal Ideation & Self-Harm'
]

# --------------------------- LangChain Model ---------------------------
model = OllamaLLM(model="llama3.2-vision:latest")

template = """
You are a mental health assistant.

Classify the following multi-turn dialog into exactly ONE topic from this list:
{topic_list}

Dialog:
{dialog}

Answer ONLY with the topic name.
"""

prompt = ChatPromptTemplate.from_template(template)
chain = prompt | model

# --------------------------- Predict Topic Function ---------------------------
def predict_topic(dialog, topics=topics):
    response = chain.invoke({
        "dialog": dialog,
        "topic_list": ", ".join(topics)
    })
    text = str(response)

    for t in topics:
        if t.lower() in text.lower():
            return t
    return "unknown"

# --------------------------- Run Example ---------------------------
if __name__ == "__main__":
    dialog = """patient: Hi, I can’t sleep at night, I keep waking up and feel exhausted during the day.
therapist: How long has this been happening?
patient: For the last two weeks, and it’s getting worse.
therapist: That sounds tough, thank you for sharing it with me."""

    pred = predict_topic(dialog)
    print(f"Dialog:\n{dialog}\n\nPredicted Topic: {pred}")

Dialog:
patient: Hi, I can’t sleep at night, I keep waking up and feel exhausted during the day.
therapist: How long has this been happening?
patient: For the last two weeks, and it’s getting worse.
therapist: That sounds tough, thank you for sharing it with me.

Predicted Topic: Sleep Disorders
